# Using EAGO's basic optimizer with user-defined subroutines 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

### Overview  
In this section, we construct an optimizer that uses EAGO's basic nlp solution routine with user-defined lower and upper bounding problems. The **EAGO.Optimizer** structure supplies a number of parameters and stored structures that advanced users may find useful for constructing specialized solution routines. For a full review, of the EAGO.optimizer object the reader is directed to the **EAGO.Optimizer** docstring and documentation provided at [https://psorlab.github.io/EAGO.jl/stable/](https://psorlab.github.io/EAGO.jl/stable/).

In this example, we'll forgo extensive integration into the EAGO.optimizer and simply replace the lower and upper-bounding problems to construct B&B routine that solves the following problem to global optimality using bounds obtained from interval arithmetic:

$
\begin{align}
&\min_{\mathbf x \in X} \;\; \sin(x_1)x_2^2 - \cos(x_3) / x_4 \label{eq1} \\
&X = [-10,10]\times[-1,1]\times[-10,10]\times[2,20].
\end{align}
$

We begin importing EAGO, IntervalArithmetic[1], and JuMP[2].

In [1]:
using EAGO, IntervalArithmetic, JuMP

### Defining a custom lower bounding problem
A valid lower bound is obtained from the lower bound of the natural interval extension using the **ValidatedNumerics.jl** package. The LowerProblem used accepts the **EAGO.Optimizer** structure and a **EAGO.NodeBB** structure, computes the bound by method overloading interval arithmetic, and stores the results to the appropriate field of the **EAGO.Optimizer's** which is a mutable structure of type **EAGO.LowerInfo**. Note that the problem is unconstrained on the domain so we can assume it is always feasible.

In [2]:
function LowerProblem!(opt::EAGO.Optimizer, n::EAGO.NodeBB)
    x = Interval.(n.lower_variable_bounds,n.upper_variable_bounds)
    FInterval = sin(x[1])x[2]^2-cos(x[3])/x[4]
    opt.current_lower_info.value = FInterval.lo
    opt.current_lower_info.solution = mid.(x)
    opt.current_lower_info.feasibility = true
end

LowerProblem! (generic function with 1 method)

### Defining a custom upper bounding problem
Since the problem is unconstained, any feasible point represents a valid upper bound. If we arbitrarily take evaluate the function at the midpoint we obtain an upper bound. If an interval extension of the midpoint is used this upper bound is guaranteed to be valid inspite of any rounding error that may occur. The below function constructs an upper bound in this manner then stores the results to the appropriate field of the **EAGO.Optimizer's** which is a mutable structure of type **EAGO.UpperInfo**.

In [3]:
function UpperProblem!(opt::EAGO.Optimizer, n::EAGO.NodeBB)
    x_value = (n.upper_variable_bounds-n.lower_variable_bounds)/2.0
    x_interval = Interval.(x_value)
    FInterval = sin(x_interval[1])x_interval[2]^2-cos(x_interval[3])/x_interval[4]
    opt.current_upper_info.value = FInterval.hi
    opt.current_upper_info.solution = x_value
    opt.current_upper_info.feasibility = true
end

UpperProblem! (generic function with 1 method)

### Build the JuMP Model and optimize
We now add our optimizer to a JuMP model, provide variable bounds, user-defined functions, and optimize. Note that options can be provided to the EAGO optimizer using a series of keywords of a Dict{Symbol,Any} object. Both manners of providing options to EAGO are illustrated below.

In [4]:
# Creates a JuMP model with the the lower_problem, upper_problem, and absolute tolerance set by keyword arguments
m = JuMP.Model(with_optimizer(EAGO.Optimizer, lower_problem! = LowerProblem!, upper_problem! = UpperProblem!, 
                              absolute_tolerance = 0.001, obbt_depth = 0, dbbt_depth = 0, cp_depth = 0,
                              treat_as_nonlinear = Int[1; 2; 3; 4]))

# Create the same model m using an options dictionary
opt_dict = Dict{Symbol, Any}()
opt_dict[:lower_problem!] = LowerProblem!
opt_dict[:upper_problem!] = UpperProblem!
opt_dict[:absolute_tolerance] = 0.001
opt_dict[:obbt_depth] = 0
opt_dict[:dbbt_depth] = 0
opt_dict[:cp_depth] = 0
opt_dict[:treat_as_nonlinear] = [1; 2; 3; 4]

m = JuMP.Model(with_optimizer(EAGO.Optimizer; opt_dict...))

# Adds variables and bounds
x_L = [-10, -1, -10, 2]
x_U = [10, 1, 10, 20]
@variable(m, x_L[i] <= x[i=1:4] <= x_U[i])

# Solves the problem
JuMP.optimize!(m)

Node ID: 1, Lower Bound: -Inf, Lower Variable Bounds:
             [-10.0, -1.0, -10.0, 2.0], Upper Variable Bounds: [10.0, 1.0, 10.0, 20.0]
started initial preprocessing
finished initial preprocessing
finished initial lower problem
finished initial upper problem
finished initial postprocessing
Lower Bound (First Iteration): -1.5000000000000007, Solution: [0.0, 0.0, 0.0, 11.0], Feasibility: true
Lower Bound (Last Iteration): -1.5000000000000007, Solution: [0.0, 0.0, 0.0, 11.0], Feasibility: true
Upper Bound: -0.45079094099198574, Solution: [10.0, 1.0, 10.0, 9.0], Feasibility: true
Iteration   NodeID    Current_LBD     Global_LBD     Global_UBD      NodesLeft     Absolute_Gap    Absolute_Ratio     LBD_Feas     UBD_Feas
     1,        2,      -1.5000000,     -Inf,     -0.4507909,           2,        Inf,       Inf,        true,       true
Node ID: 2, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-10.0, -1.0, -10.0, 2.0], Upper Variable Bounds: [0.0, 1.0, 10.0, 20

             [0.0, 0.5, 0.0, 2.0], Upper Variable Bounds: [5.0, 1.0, 5.0, 11.0]
Lower Bound (First Iteration): -1.5000000000000009, Solution: [2.5, 0.75, 2.5, 6.5], Feasibility: true
Lower Bound (Last Iteration): -1.5000000000000009, Solution: [2.5, 0.75, 2.5, 6.5], Feasibility: true
Upper Bound: 0.21543642357248266, Solution: [2.5, 0.25, 2.5, 4.5], Feasibility: true
    19,       18,      -1.5000000,     -1.5000000,     -0.8656941,          18,        0.6343059,       0.7327137,        true,       true
Node ID: 39, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [0.0, 0.5, 0.0, 6.5], Upper Variable Bounds: [5.0, 1.0, 5.0, 11.0]
Lower Bound (First Iteration): -1.1538461538461546, Solution: [2.5, 0.75, 2.5, 8.75], Feasibility: true
Lower Bound (Last Iteration): -1.1538461538461546, Solution: [2.5, 0.75, 2.5, 8.75], Feasibility: true
Upper Bound: 0.393468338138468, Solution: [2.5, 0.25, 2.5, 2.25], Feasibility: true
Iteration   NodeID    Current_LBD     Global_LBD  

             [-10.0, -1.0, -10.0, 6.5], Upper Variable Bounds: [-5.0, -0.5, -5.0, 11.0]
Lower Bound (First Iteration): -1.1538461538461546, Solution: [-7.5, -0.75, -7.5, 8.75], Feasibility: true
Lower Bound (Last Iteration): -1.1538461538461546, Solution: [-7.5, -0.75, -7.5, 8.75], Feasibility: true
Upper Bound: 0.393468338138468, Solution: [2.5, 0.25, 2.5, 2.25], Feasibility: true
    36,       33,      -1.1538462,     -1.5000000,     -0.8656941,          33,        0.6343059,       0.7327137,        true,       true
Node ID: 65, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-10.0, -0.5, -10.0, 2.0], Upper Variable Bounds: [-5.0, 0.0, 0.0, 11.0]
Lower Bound (First Iteration): -0.7500000000000003, Solution: [-7.5, -0.25, -5.0, 6.5], Feasibility: true
Lower Bound (Last Iteration): -0.7500000000000003, Solution: [-7.5, -0.25, -5.0, 6.5], Feasibility: true
Upper Bound: -0.02563153220755295, Solution: [2.5, 0.25, 5.0, 4.5], Feasibility: true
    37,       32,      

             [3.75, 0.875, 5.0, 2.0], Upper Variable Bounds: [5.0, 1.0, 6.25, 4.25]
Lower Bound (First Iteration): -1.4997247091122505, Solution: [4.375, 0.9375, 5.625, 3.125], Feasibility: true
Lower Bound (Last Iteration): -1.4997247091122505, Solution: [4.375, 0.9375, 5.625, 3.125], Feasibility: true
Upper Bound: -0.7185705700044364, Solution: [0.625, 0.0625, 0.625, 1.125], Feasibility: true
    53,       48,      -1.4997247,     -1.5000000,     -0.8656941,          48,        0.6343059,       0.7327137,        true,       true
Node ID: 83, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-10.0, 0.0, -10.0, 11.0], Upper Variable Bounds: [0.0, 1.0, 0.0, 20.0]
Lower Bound (First Iteration): -1.0909090909090915, Solution: [-5.0, 0.5, -5.0, 15.5], Feasibility: true
Lower Bound (Last Iteration): -1.0909090909090915, Solution: [-5.0, 0.5, -5.0, 15.5], Feasibility: true
Upper Bound: -0.30276710987983474, Solution: [5.0, 0.5, 5.0, 4.5], Feasibility: true
    54,       

Lower Bound (First Iteration): -1.5000000000000007, Solution: [-7.5, -0.5, 5.0, 6.5], Feasibility: true
Lower Bound (Last Iteration): -1.5000000000000007, Solution: [-7.5, -0.5, 5.0, 6.5], Feasibility: true
Upper Bound: 0.08658199481193896, Solution: [2.5, 0.5, 5.0, 4.5], Feasibility: true
Iteration   NodeID    Current_LBD     Global_LBD     Global_UBD      NodesLeft     Absolute_Gap    Absolute_Ratio     LBD_Feas     UBD_Feas
    70,       63,      -1.5000000,     -1.5000000,     -0.8656941,          63,        0.6343059,       0.7327137,        true,       true
Node ID: 141, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-10.0, -0.5, 0.0, 2.0], Upper Variable Bounds: [-5.0, 0.0, 10.0, 11.0]
Lower Bound (First Iteration): -0.7500000000000003, Solution: [-7.5, -0.25, 5.0, 6.5], Feasibility: true
Lower Bound (Last Iteration): -0.7500000000000003, Solution: [-7.5, -0.25, 5.0, 6.5], Feasibility: true
Upper Bound: -0.02563153220755295, Solution: [2.5, 0.25, 5.0, 4.5

    83,       74,      -1.4380000,     -1.5000000,     -0.8656941,          74,        0.6343059,       0.7327137,        true,       true
Node ID: 104, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-10.0, -1.0, -10.0, 2.0], Upper Variable Bounds: [-7.5, -0.75, -5.0, 6.5]
Lower Bound (First Iteration): -1.5000000000000007, Solution: [-8.75, -0.875, -7.5, 4.25], Feasibility: true
Lower Bound (Last Iteration): -1.5000000000000007, Solution: [-8.75, -0.875, -7.5, 4.25], Feasibility: true
Upper Bound: 0.3708917138094017, Solution: [1.25, 0.125, 2.5, 2.25], Feasibility: true
    84,       75,      -1.5000000,     -1.5000000,     -0.8656941,          75,        0.6343059,       0.7327137,        true,       true
Node ID: 168, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-10.0, -1.0, -10.0, 2.0], Upper Variable Bounds: [-7.5, -0.75, -7.5, 6.5]
Lower Bound (First Iteration): -1.1733176589175136, Solution: [-8.75, -0.875, -8.75, 4.25], Feasibil

Iteration   NodeID    Current_LBD     Global_LBD     Global_UBD      NodesLeft     Absolute_Gap    Absolute_Ratio     LBD_Feas     UBD_Feas
   100,       89,      -1.5000000,     -1.5000000,     -0.8656941,          89,        0.6343059,       0.7327137,        true,       true
Node ID: 201, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-2.5, 0.875, -1.25, 3.125], Upper Variable Bounds: [-1.25, 1.0, 0.0, 4.25]
Lower Bound (First Iteration): -1.3200000000000007, Solution: [-1.875, 0.9375, -0.625, 3.6875], Feasibility: true
Lower Bound (Last Iteration): -1.3200000000000007, Solution: [-1.875, 0.9375, -0.625, 3.6875], Feasibility: true
Upper Bound: -1.4394266762312964, Solution: [0.625, 0.0625, 0.625, 0.5625], Feasibility: true
   101,       40,      -1.3200000,     -1.5000000,     -1.4394267,          40,        0.0605733,       0.0420816,        true,       true
Node ID: 200, Lower Bound: -1.5000000000000007, Lower Variable Bounds:
             [-2.5, 0.875, -1.

### Get information from the JuMP Model object
The objective value, solution, termination status,  and primal status can then be accessed via the standard JuMP interface.

In [6]:
fval = JuMP.objective_value(m)
xsol = JuMP.value.(x)
status_term = JuMP.termination_status(m)
status_prim = JuMP.primal_status(m)

println("EAGO terminated with a status of $status_term and a result code of $status_prim")
println("The optimal value is: $fval, the solution found is $xsol.")

EAGO terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT
The optimal value is: -1.691376119161244, the solution found is [0.3125, 0.03125, 0.3125, 0.5625].


### Advice for more advanced constructions
The *default_lower_problem* and *default_upper_problem* should be used templates for error handling and retreiving information from MOI models. Similarly, the other default routine are good starting points for building custom modifications.

Essentially all of EAGO's subroutines are stored to a field in the **EAGO.Optimizer** structure can be reset as user-defined functions.

### References
1. IntervalArithmetic.jl [Computer software] (2019). Retrieved from https://github.com/JuliaIntervals/IntervalArithmetic.jl
2. Iain Dunning and Joey Huchette and Miles Lubin. JuMP: A Modeling Language for Mathematical Optimization, *SIAM Review*, **SIAM** 59 (2017), pp. 295-320.